<a href="https://colab.research.google.com/github/amazingashis/Hackfest/blob/master/brain_tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_recall_curve, roc_curve, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns 
plt.style.use('fivethirtyeight')
import pickle 
import os 
import numpy as np
import cv2 
%matplotlib inline
import pandas.util.testing as tm

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [ ]:
labels = ['yes', 'no']
img_size = 200
def get_training_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)


In [ ]:
data = get_training_data('/content/drive/MyDrive/Data/data/brain_tumor_dataset')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [ ]:
X = []
y = []

for feature, label in data:
    X.append(feature)
    y.append(label)



# resize data for deep learning 
X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
# good for balancing out disproportions in the dataset 
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=90, 
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=True)  

datagen.fit(X_train)

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(200,200,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(2, activation=tf.nn.softmax, name='Softmax')
])
model.summary()

testing = False
epochs = 15

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)

test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy: {}'.format(test_acc))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 99, 99, 8)         80        
_________________________________________________________________
flatten_5 (Flatten)          (None, 78408)             0         
_________________________________________________________________
Softmax (Dense)              (None, 2)                 156818    
Total params: 156,898
Trainable params: 156,898
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
7/7 [==============================] - 1s 61ms/step - loss: 0.8970 - accuracy: 0.5556
Epoch 2/15
7/7 [==============================] - 0s 61ms/step - loss: 0.7398 - accuracy: 0.5476
Epoch 3/15
7/7 [==============================] - 0s 60ms/step - loss: 0.5649 - accuracy: 0.7445
Epoch 4/15
7/7 [==============================] - 0s 61ms/step - loss: 0.3963 - accurac

In [ ]:
print(model.predict(X_test))

[[5.75040877e-01 4.24959064e-01]
 [1.44731058e-02 9.85526919e-01]
 [6.31697774e-01 3.68302166e-01]
 [9.46733534e-01 5.32664992e-02]
 [1.19393202e-03 9.98806119e-01]
 [4.33378704e-02 9.56662178e-01]
 [8.94393325e-01 1.05606705e-01]
 [9.87905622e-01 1.20944139e-02]
 [9.99058545e-01 9.41405597e-04]
 [3.28611225e-01 6.71388805e-01]
 [1.61763325e-01 8.38236630e-01]
 [8.69233191e-01 1.30766839e-01]
 [6.83307648e-02 9.31669235e-01]
 [9.77053940e-01 2.29461025e-02]
 [9.67353284e-01 3.26466933e-02]
 [9.82595444e-01 1.74045246e-02]
 [6.62962914e-01 3.37037116e-01]
 [8.54471743e-01 1.45528242e-01]
 [1.96601246e-02 9.80339944e-01]
 [7.67741919e-01 2.32258052e-01]
 [1.41304284e-01 8.58695745e-01]
 [9.88038480e-01 1.19615048e-02]
 [9.45947528e-01 5.40524237e-02]
 [7.20211327e-01 2.79788733e-01]
 [9.91300762e-01 8.69926158e-03]
 [1.41304284e-01 8.58695745e-01]
 [1.29984608e-02 9.87001538e-01]
 [1.03585962e-02 9.89641368e-01]
 [6.00623712e-03 9.93993819e-01]
 [4.48395498e-03 9.95516002e-01]
 [9.996139

In [1]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 3.8MB/s 
     |████████████████████████████████| 112kB 7.3MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [ ]:
import time
saved_model_path = "tmp/{}.h5".format(int(time.time()))

model.save(saved_model_path)

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

2021-02-20 08:42:19.514819: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
import time
model = "/content/drive/MyDrive/Data/pneumonia model/model.json"
#model.load(saved_model_path)

AttributeError: ignored